In [1]:
import pandas as pd
import numpy as np
import nltk
import tensorflow as tf
import tensorflow_addons as tfa
import keras
import torchtext

from sklearn.preprocessing import MultiLabelBinarizer
seed = 2000
np.random.seed(seed)
from tensorflow.keras import regularizers
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import BatchNormalization, Dropout, Flatten, Dense, Embedding
from tensorflow.keras.layers import LSTM, Conv1D, MaxPooling1D
from tensorflow.keras.preprocessing import sequence
from tensorflow import keras

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

import torch
from transformers import RobertaTokenizer, TFRobertaForSequenceClassification, AdamW

import warnings
warnings.filterwarnings("ignore")

In [4]:
train_sub2_df = pd.read_csv('/Users/arjunkhanchandani/Desktop/semeval2023task3bundle-v4/train_subtask_2.csv')

print(train_sub2_df.shape)
train_sub2_df.head()

dev_sub2_df = pd.read_csv('/Users/arjunkhanchandani/Desktop/semeval2023task3bundle-v4/dev_subtask_2.csv')

print(dev_sub2_df.shape)
dev_sub2_df.head()

(433, 10)
(83, 10)


,id,genre,headlines,articles,preprocessed_headlines,pos_tags_headlines,er_tags_headlines,preprocessed_articles,pos_tags_articles,er_tags_articles
0,820791520,"Political,Fairness_and_equality,Policy_prescri...",George III Lost America.\n,Theresa May Could Lose the United Kingdom Over...,george iii lose america,"[(george, 'NNP'), (iii, 'NNP'), (lose, 'VB'), ...","[(george iii, 'PERSON', 380), (america, 'GPE',...",theresa may could lose the united kingdom over...,"[(theresa, 'NN'), (may, 'MD'), (could, 'MD'), ...","[(the united kingdom, 'GPE', 384), (the europe..."
1,821040551,"Political,Capacity_and_resources,Policy_prescr...",Queen Elizabeth Would Be Evacuated in Event of...,If Britain leaves the European Union without a...,queen elizabeth would be evacuate in event of ...,"[(queen, 'NNP'), (elizabeth, 'NNP'), (would, '...","[(elizabeth, 'PERSON', 380), (brexit riot repo...",if britain leave the european union without tr...,"[(if, 'IN'), (britain, 'NNP'), (leave, 'VBP'),...","[(britain, 'GPE', 384), (the european union, '..."
2,813552066,"Public_opinion,Policy_prescription_and_evaluat...","You insult us, ambassador: Woody Johnson flagr...",With three months until Britain leaves the Eur...,you insult us ambassador woody johnson flagran...,"[(you, 'PRP'), (insult, 'VBP'), (us, 'NNP'), (...","[(woody johnson, 'PERSON', 380), (peter, 'PERS...",with three month until britain leave the europ...,"[(with, 'IN'), (three, 'CD'), (month, 'NN'), (...","[(three month, 'DATE', 391), (britain, 'GPE', ..."
3,817176202,"Political,External_regulation_and_reputation,P...","The British People, as Well as the Politicians...",The British Parliament just handed Prime Minis...,the british people as well as the politician d...,"[(the, 'DT'), (british, 'JJ'), (people, 'NNS')...","[(british, 'NORP', 381)]",the british parliament just hand prime ministe...,"[(the, 'DT'), (british, 'JJ'), (parliament, 'N...","[(british, 'NORP', 381), (british, 'NORP', 381..."
4,820419869,"Public_opinion,Political,External_regulation_a...",No break from Brexit: RT takes a look at lates...,As British MPs are told that their February br...,no break from brexit rt take look at late deve...,"[(no, 'DT'), (break, 'NN'), (from, 'IN'), (bre...","[(brexit rt, 'ORG', 383)]",as british mp be tell that their february brea...,"[(as, 'IN'), (british, 'NNP'), (mp, 'NNP'), (b...","[(british, 'NORP', 381), (february, 'DATE', 39..."


In [5]:
train_sub2_df["genre"] = train_sub2_df["genre"].str.split(',')
dev_sub2_df["genre"] = dev_sub2_df["genre"].str.split(',')

In [6]:
train_sub2_df["genre"]

0      [Political, Morality, Fairness_and_equality, E...
1      [Morality, Security_and_defense, Policy_prescr...
2      [Political, Crime_and_punishment, External_reg...
3      [Political, Crime_and_punishment, Fairness_and...
4      [Political, Morality, Fairness_and_equality, E...
                             ...                        
428    [Political, Crime_and_punishment, External_reg...
429    [Political, Morality, Crime_and_punishment, Qu...
430    [Crime_and_punishment, Political, Legality_Con...
431    [Political, Crime_and_punishment, Fairness_and...
432    [Crime_and_punishment, Political, Legality_Con...
Name: genre, Length: 433, dtype: object

In [7]:
dev_sub2_df["genre"]

0     [Political, Fairness_and_equality, Policy_pres...
1     [Political, Capacity_and_resources, Policy_pre...
2     [Public_opinion, Policy_prescription_and_evalu...
3     [Political, External_regulation_and_reputation...
4     [Public_opinion, Political, External_regulatio...
                            ...                        
78    [Political, Crime_and_punishment, Fairness_and...
79    [Policy_prescription_and_evaluation, Political...
80    [Policy_prescription_and_evaluation, Political...
81    [Crime_and_punishment, Political, Legality_Con...
82    [Political, Morality, Crime_and_punishment, Qu...
Name: genre, Length: 83, dtype: object

In [8]:
mlb = MultiLabelBinarizer()
#multi_train = pd.DataFrame(mlb.fit_transform(train_sub2_df['genre']), columns = mlb.classes_)
multi_train = mlb.fit_transform(train_sub2_df['genre'])
print(train_sub2_df.genre.value_counts())

mlb = MultiLabelBinarizer()
#multi_dev = pd.DataFrame(mlb.fit_transform(dev_sub2_df['genre']), columns = mlb.classes_)
multi_dev = mlb.fit_transform(dev_sub2_df['genre'])
print(dev_sub2_df.genre.value_counts())

[Health_and_safety, Quality_of_life]                                                                                                                    11
[Political, Crime_and_punishment, Morality, Legality_Constitutionality_and_jurisprudence]                                                               10
[Political, Crime_and_punishment, Legality_Constitutionality_and_jurisprudence]                                                                          9
[Crime_and_punishment, Security_and_defense, Legality_Constitutionality_and_jurisprudence]                                                               8
[Crime_and_punishment, Morality]                                                                                                                         8
                                                                                                                                                        ..
[Crime_and_punishment, Morality, Fairness_and_equality, Security_and_d

In [9]:
multi_train

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 1, 0, ..., 1, 0, 0],
       ...,
       [0, 1, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0]])

In [10]:
multi_dev

array([[0, 0, 0, ..., 1, 0, 1],
       [1, 0, 0, ..., 1, 0, 1],
       [0, 0, 0, ..., 1, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 1, 0]])

In [11]:
x_train = train_sub2_df.preprocessed_articles + train_sub2_df.preprocessed_headlines
y_train = multi_train

x_dev = dev_sub2_df.preprocessed_articles + dev_sub2_df.preprocessed_headlines
y_dev = multi_dev

# x_train = train_sub1_df.text
# y_train = train_sub1_df.target

# x_test = train_sub1_df.text
tokenizer = RobertaTokenizer.from_pretrained('roberta-base', num_labels=14, output_attentions=True)

In [12]:
x_train = x_train.to_numpy().reshape(-1)

x_dev = x_dev.to_numpy().reshape(-1)

# x_test = x_test.to_numpy().reshape(-1)

In [13]:
def roberta_encode(texts, tokenizer):
    MAX_LEN = 512
    ct = len(texts)
    input_ids = np.ones((ct, MAX_LEN), dtype='int32')
    attention_mask = np.zeros((ct, MAX_LEN), dtype='int32')
    token_type_ids = np.zeros((ct, MAX_LEN), dtype='int32') # Not used in text classification

    for k, text in enumerate(texts):
        # Tokenize
        tok_text = tokenizer.tokenize(str(text))
        
        # Truncate and convert tokens to numerical IDs
        enc_text = tokenizer.convert_tokens_to_ids(tok_text[:(MAX_LEN-2)])
        
        input_length = len(enc_text) + 2
        input_length = input_length if input_length < MAX_LEN else MAX_LEN
        
        # Add tokens [CLS] and [SEP] at the beginning and the end
        input_ids[k,:input_length] = np.asarray([0] + enc_text + [2], dtype='int32')
        
        # Set to 1s in the attention input
        attention_mask[k,:input_length] = 1

    return {
        'input_word_ids': input_ids,
        'input_mask': attention_mask,
        'input_type_ids': token_type_ids
    }

In [14]:
x_train = roberta_encode(x_train, tokenizer)
x_dev = roberta_encode(x_dev, tokenizer)

y_train = np.asarray(y_train, dtype='int32')
y_dev = np.asarray(y_dev, dtype='int32')

In [15]:
def build_model(n_categories):
    MAX_LEN = 512
    input_word_ids = tf.keras.Input(shape=(MAX_LEN,), dtype=tf.int32, name='input_word_ids')
    input_mask = tf.keras.Input(shape=(MAX_LEN,), dtype=tf.int32, name='input_mask')
    input_type_ids = tf.keras.Input(shape=(MAX_LEN,), dtype=tf.int32, name='input_type_ids')

    # Import RoBERTa model from HuggingFace
    roberta_model = TFRobertaForSequenceClassification.from_pretrained('roberta-base')
    # custom_objects = {"TFRobertaForSequenceClassification": TFRobertaForSequenceClassification}
    # config = roberta_model.get_config()
    # with tf.keras.utils.custom_object_scope(custom_objects):
    #     roberta_model = TFRobertaForSequenceClassification.from_config(config)
    x = roberta_model(input_word_ids, attention_mask=input_mask, token_type_ids=input_type_ids)

    # Huggingface transformers have multiple outputs, embeddings are the first one,
    # so let's slice out the first position
    x = x[0]

    x = tf.keras.layers.Dropout(rate=0.1)(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(256, activation='relu')(x)
    x = tf.keras.layers.Dense(n_categories, activation='sigmoid')(x)

    model = tf.keras.Model(inputs=[input_word_ids, input_mask, input_type_ids], outputs=x)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001),
        loss='binary_crossentropy',
        metrics=['accuracy'])
    
    

    return model

In [16]:
model = build_model(14)
model.summary()

2023-01-28 02:37:36.497162: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-01-28 02:37:36.497621: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_word_ids (InputLayer)    [(None, 512)]        0           []                               
                                                                                                  
 input_mask (InputLayer)        [(None, 512)]        0           []                               
                                                                                                  
 input_type_ids (InputLayer)    [(None, 512)]        0           []                               
                                                                                                  
 tf_roberta_for_sequence_classi  TFSequenceClassifie  124647170  ['input_word_ids[0][0]',         
 fication (TFRobertaForSequence  rOutput(loss=None,               'input_mask[0][0]',         

In [17]:
model.fit(x_train, y_train, epochs=50, batch_size=4, validation_data=(x_dev, y_dev))

Epoch 1/50


2023-01-28 02:38:19.294910: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-01-28 02:38:24.593509: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


109/109 [==============================] - ETA: 0s - loss: 0.6928 - accuracy: 0.0647

2023-01-28 02:44:17.813464: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


109/109 [==============================] - 378s 3s/step - loss: 0.6928 - accuracy: 0.0647 - val_loss: 0.6916 - val_accuracy: 0.0241
Epoch 2/50
109/109 [==============================] - 357s 3s/step - loss: 0.6827 - accuracy: 0.0531 - val_loss: 0.6571 - val_accuracy: 0.0241
Epoch 3/50
109/109 [==============================] - 339s 3s/step - loss: 0.6463 - accuracy: 0.0531 - val_loss: 0.6307 - val_accuracy: 0.0241
Epoch 4/50
109/109 [==============================] - 357s 3s/step - loss: 0.6027 - accuracy: 0.0531 - val_loss: 0.6121 - val_accuracy: 0.0241
Epoch 5/50
109/109 [==============================] - 373s 3s/step - loss: 0.5690 - accuracy: 0.0531 - val_loss: 0.6046 - val_accuracy: 0.0241
Epoch 6/50
109/109 [==============================] - 355s 3s/step - loss: 0.5426 - accuracy: 0.0531 - val_loss: 0.6030 - val_accuracy: 0.0241
Epoch 7/50
109/109 [==============================] - 395s 4s/step - loss: 0.5234 - accuracy: 0.0924 - val_loss: 0.5988 - val_accuracy: 0.0120
Epoch 8/50

In [ ]:
y_pred = model.predict(x_dev)

In [ ]:
y_pred

In [ ]:
y_pred_final = list()
y_pred_sub = list()
for val in y_pred:
    if val[0]>val[1] and val[0]>val[2]:
        y_pred_final.append(0)
        y_pred_sub.append('opinion')
    elif val[1]>val[0] and val[1]>val[2]:
        y_pred_final.append(1)
        y_pred_sub.append('reporting')
    else:
        y_pred_final.append(2)
        y_pred_sub.append('satire')

print(y_pred_final)  
print(list(y_dev))  

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[array([0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1], dtype=int32), array([1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1], dtype=int32), array([0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0], dtype=int32), array([0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0], dtype=int32), array([1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0], dtype=int32), array([0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0], dtype=int32), array([0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1], dtype=int32), array([0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0], dtype=int32), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0], dtype=int32), array([0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1], dtype=int32), array([0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0], dtype=int32), array([0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0,

In [ ]:
from sklearn.metrics import f1_score
print('Macro f1_score = {}'.format(f1_score(y_pred_final, y_dev, average='macro')))
print('Micro f1_score = {}'.format(f1_score(y_pred_final, y_dev, average='micro')))

ValueError: ignored

In [ ]:
y_dev_id = dev_sub1_df.id

In [ ]:
dict = {'article_id': y_dev_id, 'class': y_pred_sub} 
submit_df = pd.DataFrame(dict) 
submit_df.set_index('article_id', inplace=True)
submit_df.head()

In [ ]:
submit_df.to_csv('data.txt', sep='\t')